## FX Strategy Example

In [1]:
import os

os.chdir("..")

In [2]:
from datetime import date
from pxtrade import Trade, Strategy, History, Backtest
from pxtrade.assets import reset, Cash, FxRate, Portfolio
from pxtrade.events.yahoo import load_yahoo_prices
from pxtrade.compliance import Compliance, UnitLimit

In [3]:
# initialise assets
reset()
aud = Cash("AUD")
usd = Cash("USD")
audusd = FxRate("AUDUSD")
portfolio = Portfolio("USD")
starting_value = int(1e6)
portfolio.transfer(usd, starting_value)


In [4]:
# impose limits (either short or long) for AUD
limit = int(2e6)

portfolio.compliance = Compliance().add_rule(
    UnitLimit(aud, limit)
)

In [5]:
class AUDStrategy(Strategy):
    def generate_trades(self):
        audusd_rate = audusd.rate
        trade_amount = 10000

        if audusd_rate <= 0.65:
            return Trade(portfolio, aud, +trade_amount)

        if audusd_rate >= 0.95:
            return Trade(portfolio, aud, -trade_amount)


In [6]:
backtest = Backtest(AUDStrategy())
history = History(portfolio)

In [7]:
start_date = date(2010, 1, 1)
end_date = date(2020, 10, 1)
load_yahoo_prices(
    audusd,
    backtest,
    start_date=start_date,
    end_date=end_date,
)

In [8]:
backtest.run()

In [9]:
df = history.get()

df

,AUD,AUDUSD,Portfolio,Portfolio_AUD,Portfolio_USD,USD
2010-01-01,1.0,0.898473,1.000000e+06,0.0,1.000000e+06,1.0
2010-01-04,1.0,0.912492,1.000000e+06,0.0,1.000000e+06,1.0
2010-01-05,1.0,0.911826,1.000000e+06,0.0,1.000000e+06,1.0
2010-01-06,1.0,0.920471,1.000000e+06,0.0,1.000000e+06,1.0
2010-01-07,1.0,0.917768,1.000000e+06,0.0,1.000000e+06,1.0
...,...,...,...,...,...,...
2020-09-24,1.0,0.705716,1.658236e+06,-1550000.0,2.752097e+06,1.0
2020-09-27,1.0,0.704191,1.660601e+06,-1550000.0,2.752097e+06,1.0
2020-09-28,1.0,0.709522,1.652338e+06,-1550000.0,2.752097e+06,1.0
2020-09-29,1.0,0.714362,1.644835e+06,-1550000.0,2.752097e+06,1.0


In [10]:
import cufflinks as cf

columns = ["Portfolio", "Portfolio_AUD", "Portfolio_USD", "AUDUSD"]

df[columns].iplot(
    secondary_y="AUDUSD",
    title="Portfolio Value",
)

***